In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('../')

In [4]:
import sys
sys.path.append('/home/njr61/rds/hpc-work/spurious-concepts/ConceptBottleneck')

In [5]:
import torch
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
import torch.nn as nn
import torch.optim as optim
import pickle
import matplotlib.pyplot as plt
import torch.nn.functional as F
from PIL import Image
from captum.attr import visualization as viz
from matplotlib.colors import LinearSegmentedColormap
import cv2
from copy import copy 
import itertools
import json

In [6]:
from ConceptBottleneck.CUB.models import ModelXtoC, ModelOracleCtoY
from ConceptBottleneck.CUB.dataset import load_data

In [7]:
from src.images import *
from src.util import *
from src.models import *
from src.plot import *

## Impact of Bounding Box Size

### Small Model

In [8]:
num_objects = 1
noisy=False
weight_decay = 0.0004
encoder_model='small3'
optimizer = 'sgd'
seed = 42

In [10]:
json_file = "results/ablation/bounding_box.json"
results_dict = {
    'num_objects': num_objects, 
    'noisy': noisy,
    'weight_decay': weight_decay,
    'encoder_model': encoder_model, 
    'optimizer': optimizer, 
    'seed': seed,
}

In [11]:
np.random.seed(seed)
torch.manual_seed(seed)

In [12]:
train_loader, val_loader, train_pkl, val_pkl = get_data(num_objects, noisy)
val_images, val_y, val_c = unroll_data(val_loader)

In [13]:
joint_model = get_synthetic_model(num_objects,encoder_model,noisy,weight_decay,optimizer,seed)

In [14]:
all_epsilons = {}

for epsilon in range(0,51,10):
    activation_values = []

    for concept_num in range(num_objects*2):
        ret_image = get_maximal_activation(joint_model,run_joint_model,concept_num,
                                        get_valid_image_function(concept_num,num_objects,epsilon=epsilon))
        activation_values.append (
            torch.nn.Sigmoid()(run_joint_model(joint_model,ret_image)[1])[concept_num][0].detach().numpy()
        )

    all_epsilons[epsilon] = activation_values
        

In [ ]:
results_dict['epsilon_adversarial'] = all_epsilons

In [22]:
for i in results_dict['epsilon_adversarial']:
    results_dict['epsilon_adversarial'][i] = np.array(results_dict['epsilon_adversarial'][i]).tolist() 

In [23]:
json.dump(results_dict,open(json_file,"w"))

In [24]:
results_dict

{'num_objects': 1,
 'noisy': False,
 'weight_decay': 0.0004,
 'encoder_model': 'small3',
 'optimizer': 'sgd',
 'seed': 42,
 'epsilon_adversarial': {0: [1.0, 1.0],
  10: [1.0, 1.0],
  20: [1.0, 0.07281245291233063],
  30: [1.0, 0.0015784641727805138],
  40: [1.0, 1.1123177500849124e-05],
  50: [1.0, 5.825044269158752e-08]}}

### Large Model

In [7]:
num_objects = 1
noisy=False
weight_decay = 0.0004
encoder_model='small7'
optimizer = 'sgd'
seed = 42

In [8]:
json_file = "results/ablation/bounding_box_large.json"
results_dict = {
    'num_objects': num_objects, 
    'noisy': noisy,
    'weight_decay': weight_decay,
    'encoder_model': encoder_model, 
    'optimizer': optimizer, 
    'seed': seed,
}

In [9]:
np.random.seed(seed)
torch.manual_seed(seed)

In [10]:
train_loader, val_loader, train_pkl, val_pkl = get_data(num_objects, noisy)
val_images, val_y, val_c = unroll_data(val_loader)

In [11]:
joint_model = get_synthetic_model(num_objects,encoder_model,noisy,weight_decay,optimizer,seed)

In [12]:
all_epsilons = {}

for epsilon in range(0,51,10):
    activation_values = []

    for concept_num in range(num_objects*2):
        ret_image = get_maximal_activation(joint_model,run_joint_model,concept_num,
                                        get_valid_image_function(concept_num,num_objects,epsilon=epsilon))
        activation_values.append (
            torch.nn.Sigmoid()(run_joint_model(joint_model,ret_image)[1])[concept_num][0].detach().numpy()
        )

    all_epsilons[epsilon] = activation_values
        

In [13]:
results_dict['epsilon_adversarial'] = all_epsilons

In [14]:
for i in results_dict['epsilon_adversarial']:
    results_dict['epsilon_adversarial'][i] = np.array(results_dict['epsilon_adversarial'][i]).tolist() 

In [15]:
json.dump(results_dict,open(json_file,"w"))

In [16]:
results_dict

{'num_objects': 1,
 'noisy': False,
 'weight_decay': 0.0004,
 'encoder_model': 'small7',
 'optimizer': 'sgd',
 'seed': 42,
 'epsilon_adversarial': {0: [1.0, 1.0],
  10: [1.0, 1.0],
  20: [1.0, 1.0],
  30: [1.0, 1.0],
  40: [1.0, 1.0],
  50: [1.0, 1.0]}}

## Pruning Ablation Study

### Weight Pruning

In [8]:
num_objects = 1
noisy=False
weight_decay = 0.0004
encoder_model='small7'
optimizer = 'sgd'
seed = 42

results_json = "results/ablation/pruning.json"
results = {
    'layer': {},
    'weight': {},
    'num_objects': num_objects, 
    'noisy': noisy,
    'weight_decay': weight_decay,
    'encoder_model': encoder_model, 
    'optimizer': optimizer, 
    'seed': seed,
}

In [9]:
np.random.seed(seed)
torch.manual_seed(seed)

In [10]:
train_loader, val_loader, train_pkl, val_pkl = get_data(num_objects, noisy)
val_images, val_y, val_c = unroll_data(val_loader)

In [11]:
no_color = torch.Tensor([0.25,0.25,0.25])
full_color = torch.Tensor([-0.25,-0.25,0.25])

In [12]:
blank_image = no_color.clone().view(3, 1, 1).expand((3,256,256))
full_image = full_color.clone().view(3, 1, 1).expand_as(blank_image)

half_left = no_color.view(3, 1, 1).expand_as(blank_image).clone()
half_left[:,:,:128] = full_image[:,:,:128]

half_right = no_color.clone().view(3, 1, 1).expand_as(blank_image).clone()
half_right[:,:,128:] = full_image[:,:,128:]

In [17]:
for prune_rate in [0.25,0.5,0.75,0.9,0.95,0.99]:
    joint_model = get_synthetic_model(num_objects,encoder_model,noisy,weight_decay,optimizer,seed)

    for layer in joint_model.first_model.children():
        if isinstance(layer, torch.nn.Conv2d) or isinstance(layer, torch.nn.Linear):
            torch.nn.utils.prune.l1_unstructured(layer, name="weight", amount=prune_rate)

    activation_values = []

    for concept_num in range(num_objects*2):
        ret_image = get_maximal_activation(joint_model,run_joint_model,concept_num,
                                        get_valid_image_function(concept_num,num_objects,epsilon=32))
        activation_values.append (
        torch.nn.Sigmoid()(run_joint_model(joint_model,ret_image)[1])[concept_num][0].detach().numpy()
    )

    results['weight'][prune_rate] = {
        'activations': np.array(activation_values).tolist(), 
        'accuracies': np.array(get_concept_accuracy_by_concept(joint_model,run_joint_model,train_loader,sigmoid=True).detach().numpy()).tolist(), 
    }
    

### Layer Pruning

In [18]:
for prune_rate in [0.25,0.5,0.75,0.9,0.95,0.99]:
    joint_model = get_synthetic_model(num_objects,encoder_model,noisy,weight_decay,optimizer,seed)

    for layer_to_prune in [joint_model.first_model.conv4, 
                        joint_model.first_model.conv5,
                        joint_model.first_model.conv6,
                        joint_model.first_model.conv7]:

        weight = layer_to_prune.weight.data.abs().clone()
        importance = weight.sum((1, 2, 3))  # Calculate importance of filters
        num_filters = layer_to_prune.weight.size(0)

        # Compute the number of filters to prune
        num_prune = int(num_filters * prune_rate)
        _, indices = importance.sort(descending=True)
        indices_to_prune = indices[-num_prune:]

        # Create a mask to prune filters
        mask = torch.ones(num_filters)
        mask[indices_to_prune] = 0
        if mask is not None:
            mask = mask.to(layer_to_prune.weight.device)
            layer_to_prune.weight.data *= mask.view(-1, 1, 1, 1)


    activation_values = []

    for concept_num in range(num_objects*2):
        ret_image = get_maximal_activation(joint_model,run_joint_model,concept_num,
                                        get_valid_image_function(concept_num,num_objects,epsilon=32))
        activation_values.append (
        torch.nn.Sigmoid()(run_joint_model(joint_model,ret_image)[1])[concept_num][0].detach().numpy()
    )

    results['layer'][prune_rate] = {
        'activations': np.array(activation_values).tolist(), 
        'accuracies': np.array(get_concept_accuracy_by_concept(joint_model,run_joint_model,train_loader,sigmoid=True).detach().numpy()).tolist(), 
    }


    

In [19]:
json.dump(results,open(results_json,'w'))

: 